# Adaptation of the notebook matching-faces.ipynb

From: https://github.com/emmanuel-hartman/H2_SurfaceMatch/blob/main/demo/matching-faces.ipynb

## Set up

In [1]:
import os
import subprocess

gitroot_path = subprocess.check_output(
    ["git", "rev-parse", "--show-toplevel"], universal_newlines=True
)
os.chdir(gitroot_path[:-1])
print("Working directory: ", os.getcwd())

import warnings

warnings.filterwarnings("ignore")

import sys

sys_dir = os.path.dirname(os.getcwd())
sys.path.append(sys_dir)
print("Directory added to path: ", sys_dir)
h2_surfacematch_dir = os.path.join(sys_dir, "H2_SurfaceMatch")
sys.path.append(h2_surfacematch_dir)
print("H2_SurfaceMatch directory added to path: ", h2_surfacematch_dir)

Working directory:  /home/nmiolane/code/my28brains
Directory added to path:  /home/nmiolane/code
H2_SurfaceMatch directory added to path:  /home/nmiolane/code/H2_SurfaceMatch


In [2]:
import sys

sys.path.append("..")
import H2_match as gm
import numpy as np
import SRNF_match
import torch
import utils.input_output as input_output

use_cuda = 1
torchdeviceId = torch.device("cuda:0") if use_cuda else "cpu"
torchdtype = torch.float32

[KeOps] Warning : cuda was detected, but driver API could not be initialized. Switching to cpu only.
Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
base_path = "/home/nmiolane/code/my28brains/data/registered_meshes/left_structure_-1_sub-01_ses-02.ply"
one_path = "/home/nmiolane/code/my28brains/data/registered_meshes/left_structure_-1_sub-01_ses-03.ply"

N = 2
[VS, FS, FunS] = input_output.loadData(
    base_path
)  # "TestData/faces/face-reference.ply")
VS = VS / 10
[VS, FS] = input_output.decimate_mesh(VS, FS, int(FS.shape[0] / 16))
FunS = np.zeros((int(np.size(VS) / 3),))
sources = [[VS, FS]]

[VT, FT, FunT] = input_output.loadData(one_path)  # "TestData/faces/face-rage.ply")
VT = VT / 10
[VT, FT] = input_output.decimate_mesh(VT, FT, int(FT.shape[0] / 16))
FunT = np.zeros((int(np.size(VT) / 3),))
targets = [[VT, FT]]

source = sources[0]
target = targets[0]

In [4]:
a0 = 0.01
a1 = 100
b1 = 100
c1 = 0.2
d1 = 0.01
a2 = 0.01


param1 = {
    "weight_coef_dist_T": 10**1,
    "weight_coef_dist_S": 10**1,
    "sig_geom": 0.4,
    "max_iter": 2000,
    "time_steps": 2,
    "tri_unsample": True,
    "index": 0,
}

param2 = {
    "weight_coef_dist_T": 10**2,
    "weight_coef_dist_S": 10**2,
    "sig_geom": 0.3,
    "max_iter": 1000,
    "time_steps": 2,
    "tri_unsample": False,
    "index": 1,
}

param3 = {
    "weight_coef_dist_T": 10**3,
    "weight_coef_dist_S": 10**3,
    "sig_geom": 0.2,
    "max_iter": 1000,
    "time_steps": 2,
    "tri_unsample": False,
    "index": 1,
}

param4 = {
    "weight_coef_dist_T": 10**4,
    "weight_coef_dist_S": 10**4,
    "sig_geom": 0.1,
    "max_iter": 1000,
    "time_steps": 3,
    "tri_unsample": True,
    "index": 1,
}

param5 = {
    "weight_coef_dist_T": 10**5,
    "weight_coef_dist_S": 10**5,
    "sig_geom": 0.1,
    "max_iter": 1000,
    "time_steps": 4,
    "tri_unsample": False,
    "index": 2,
}

param6 = {
    "weight_coef_dist_T": 10**6,
    "weight_coef_dist_S": 10**6,
    "sig_geom": 0.05,
    "max_iter": 1000,
    "time_steps": 5,
    "tri_unsample": False,
    "index": 2,
}


paramlist = [param1, param2, param3, param4, param5, param6]

In [6]:
# # started around 8.45 am
# geod, F0 = gm.H2MultiRes(
#     source, target, a0, a1, b1, c1, d1, a2, resolutions=2, paramlist=paramlist
# )
import pickle

NOTEBOOKS_DIR = os.path.join(os.getcwd(), "my28brains", "notebooks")
geod_pkl = os.path.join(NOTEBOOKS_DIR, 'geod.pickle')
F0_pkl = os.path.join(NOTEBOOKS_DIR, 'F0.pickle')


with open(geod_pkl, 'rb') as handle:
    geod = pickle.load(handle)
    
with open(F0_pkl, 'rb') as handle:
    F0 = pickle.load(handle)

In [7]:
# import pickle


# with open(geod_pkl, 'wb') as handle:
#     pickle.dump(geod, handle, protocol=pickle.HIGHEST_PROTOCOL)

# assert np.all(geod == geod_loaded)

# with open(F0_pkl, 'wb') as handle:
#     pickle.dump(F0, handle, protocol=pickle.HIGHEST_PROTOCOL)


    
# assert np.all(F0 == F0_loaded)

In [8]:
# import importlib
# from H2_SurfaceMatch.utils import utils
# import H2_SurfaceMatch.utils
# import H2_SurfaceMatch.utils.utils
# import H2_SurfaceMatch
# importlib.reload(H2_SurfaceMatch)
# importlib.reload(input_output)

# importlib.reload(H2_SurfaceMatch.utils)
# importlib.reload(H2_SurfaceMatch.utils.utils)

In [11]:
result_ply = os.path.join(NOTEBOOKS_DIR, "left_h2match")

input_output.plotGeodesic(
    geod,
    F0,
    source=source,
    target=target,
    file_name=result_ply,
    stepsize=2,
    axis=[0, 0, 1],
    angle=-1 * np.pi,
)

[Open3D WARNING] GLFW Error: X11: The DISPLAY environment variable is missing
[Open3D WARNING] Failed to initialize GLFW
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.


In [17]:
len(geod)

60

In [18]:
import numpy as np
import scipy

gif_file_name = os.path.join(NOTEBOOKS_DIR, "gif", "left_h2match")

#xp = np.linspace(0, 1, 27, endpoint=True)
x = np.linspace(0, 1, 60, endpoint=True)
#f = scipy.interpolate.interp1d(xp, geod, axis=0)
#geod = f(x)
for i in range(0, geod.shape[0]):
    input_output.plotGeodesic(
        [geod[i]],
        F0,
        stepsize=5,
        file_name=gif_file_name  +"{}".format(i),
        axis=[0, 1, 0],
        angle=-1 * np.pi / 2,
    )

[Open3D WARNING] GLFW Error: X11: The DISPLAY environment variable is missing
[Open3D WARNING] Failed to initialize GLFW
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.
[Open3D WARNING] GLFW Error: X11: The DISPLAY environment variable is missing
[Open3D WARNING] Failed to initialize GLFW
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.
[Open3D WARNING] GLFW Error: X11: The DISPLAY environment variable is missing
[Open3D WARNING] Failed to initialize GLFW
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.
[Open3D WARNING] GLFW Error: X11: The DISPLAY environment variable is missing
[Open3D WARNING] Failed to initialize GLFW
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.
[Open3D WARNING] GLFW Error: X11: The DISPLAY environment variable is missing
[Open3D WARNING] Failed to initialize GLFW
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.
[Open3D WARNING] GLFW Error: X11: The DISPLAY environment variable is 

In [15]:
np.linspace(0, 1, 5, endpoint=True)

array([0.  , 0.25, 0.5 , 0.75, 1.  ])